In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd

In [ ]:
# Define paths to the directories
train_dir = 'C:\\Users\\Rajabi\\Desktop\\NASA\\RSSCN7-master\\New\\train'
validation_dir = 'C:\\Users\\Rajabi\\Desktop\\NASA\\RSSCN7-master\\New\\validation'
test_dir = 'C:\\Users\\Rajabi\\Desktop\\NASA\\RSSCN7-master\\New\\test'

In [ ]:
# Define image data generators
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Specify the batch size and target image size
batch_size = 32
target_size = (400, 400)

In [ ]:
# Create generators for training, validation, and test sets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
# Define hyperparameters to experiment with
conv2d_layers = [1, 2]
maxpooling2d_layers = [1, 2]
dense_layers = [1, 2]
neurons_per_layer = [64, 128]
batch_sizes = [16, 32]
learning_rates = [0.0001, 0.0005]
activation_functions = ['relu', 'sigmoid']

In [ ]:
# Perform experiments and store results in a list of dictionaries
results = []

for conv2d in conv2d_layers:
    for maxpooling2d in maxpooling2d_layers:
        for dense in dense_layers:
            for neurons in neurons_per_layer:
                for batch_size in batch_sizes:
                    for lr in learning_rates:
                        for activation in activation_functions:
                            # Create CNN model with current hyperparameters
                            model = Sequential()
                            model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(400, 400, 3)))
                            for _ in range(conv2d - 1):
                                model.add(Conv2D(64, (3, 3), activation='relu'))
                            model.add(MaxPooling2D((2, 2)))
                            for _ in range(maxpooling2d - 1):
                                model.add(MaxPooling2D((2, 2)))
                            model.add(Flatten())
                            for _ in range(dense):
                                model.add(Dense(neurons, activation=activation))
                            model.add(Dense(7, activation='softmax'))

                            model.compile(optimizer=tf.keras.optimizers.Adam(lr=lr),
                                          loss='categorical_crossentropy',
                                          metrics=['accuracy'])

                            # Train the model
                            history = model.fit(
                                train_generator,
                                epochs=20,
                                validation_data=validation_generator,
                                verbose=0
                            )

                            # Evaluate the model on test set
                            test_loss, test_acc = model.evaluate(test_generator, verbose=0)

                            # Store results in a list
                            results.append({
                                'Conv2D Layers': conv2d,
                                'MaxPooling2D Layers': maxpooling2d,
                                'Dense Layers': dense,
                                'Neurons per Dense Layer': neurons,
                                'Batch Size': batch_size,
                                'Learning Rate': lr,
                                'Activation Function': activation,
                                'Test Accuracy': test_acc
                            })